<a href="https://colab.research.google.com/github/folaAj/AdventureWorks/blob/master/NLP_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets
import pyarrow
if int(pyarrow.__version__.split('.')[1]) < 16 and int(pyarrow.__version__.split('.')[0]) == 0:
    import os
    os.kill(os.getpid(), 9)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install -q -U "tensorflow-text==2.8.*"

In [4]:
!pip install -q tf-models-official==2.7.0

In [5]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import unicodedata

from datasets import load_dataset
from transformers import AutoTokenizer

tf.get_logger().setLevel('ERROR')

# Read dataset

In [6]:
dataset = load_dataset("rotten_tomatoes")

  0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})

In [9]:
from tokenizers import normalizers
from tokenizers.normalizers import NFKD, StripAccents 
normalizer = normalizers.Sequence([NFKD(), StripAccents()])

In [10]:
BATCH_SIZE = 32
def normalize(x):
  x['text'] = unicodedata.normalize('NFKD', x['text']).encode('ascii', 'ignore')
  return x
train = dataset['train'].map(normalize).to_tf_dataset(batch_size=BATCH_SIZE, shuffle=True).map(lambda x: (x['text'], x['label']))
val = dataset['validation'].map(normalize).to_tf_dataset(batch_size=BATCH_SIZE, shuffle=True).map(lambda x: (x['text'], x['label']))
test = dataset['test'].map(normalize).to_tf_dataset(batch_size=BATCH_SIZE, shuffle=True).map(lambda x: (x['text'], x['label']))

# Finetune  a BERT Model For Sentiment Analysis on Rotten Tomaotoes Dataset.

In [11]:
def build_classifier_model(should_finetune):
  tfhub_handle_encoder = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1'
  tfhub_handle_preprocess =  'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=should_finetune, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
  return tf.keras.Model(text_input, net)

In [12]:
strategy = tf.distribute.MirroredStrategy()

In [13]:
with strategy.scope():
  metrics = tf.metrics.BinaryAccuracy()
  loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
  optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)  
  bert_finetuned_model = build_classifier_model(should_finetune=True)
  bert_finetuned_model.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)

In [14]:
bert_finetuned_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 preprocessing (KerasLayer)     {'input_type_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128)}                                                  

In [15]:
EPOCH = 10
bert_finetuned_model.fit(x=train, validation_data=val, epochs=EPOCH)

Epoch 1/10
267/267 [==============================] - 72s 240ms/step - loss: 0.5772 - binary_accuracy: 0.6733 - val_loss: 0.4987 - val_binary_accuracy: 0.7514
Epoch 2/10
267/267 [==============================] - 61s 230ms/step - loss: 0.4627 - binary_accuracy: 0.7761 - val_loss: 0.4387 - val_binary_accuracy: 0.7899
Epoch 3/10
267/267 [==============================] - 64s 242ms/step - loss: 0.3892 - binary_accuracy: 0.8216 - val_loss: 0.4299 - val_binary_accuracy: 0.7974
Epoch 4/10
267/267 [==============================] - 64s 238ms/step - loss: 0.3420 - binary_accuracy: 0.8450 - val_loss: 0.4268 - val_binary_accuracy: 0.8049
Epoch 5/10
267/267 [==============================] - 64s 240ms/step - loss: 0.2834 - binary_accuracy: 0.8750 - val_loss: 0.4744 - val_binary_accuracy: 0.8039
Epoch 6/10
267/267 [==============================] - 63s 236ms/step - loss: 0.2281 - binary_accuracy: 0.8984 - val_loss: 0.4846 - val_binary_accuracy: 0.8096
Epoch 7/10
267/267 [==========================